In [ ]:
%load_ext autoreload
%autoreload 2


import logging


logging.basicConfig(
    format='%(asctime)s %(name)s %(levelname)-8s %(message)s',
    level=logging.DEBUG,
    datefmt='%Y-%m-%d %H:%M:%S')

logger = logging.getLogger("fab_duckdb")
logger.setLevel(logging.DEBUG)



In [ ]:
import duckdb
import fabduckdb

createstatement = "CREATE or REPLACE TABLE abc as SELECT * FROM range(1,10,1) t(x);CREATE or REPLACE TABLE def as SELECT * FROM range(20,30,2) t(x)"
loopstatement = "loop (select * from {{x}}) over (select x from (values('abc'), ('def')) t(x))"
duckdb.execute(createstatement)
duckdb.execute(loopstatement).df()

In [ ]:
import duckdb
import fabduckdb

createstatement = "CREATE or REPLACE TABLE abc as SELECT * FROM range(1,10,1) t(x);CREATE or REPLACE TABLE def as SELECT * FROM range(20,30,2) t(x)"
loopstatement = "loop (select * from {{x}}) over (select x from (values('abc'), ('def')) t(x))"
duckdb.default_connection.execute(createstatement)
duckdb.default_connection.execute(loopstatement).df()

In [ ]:
# Embedded Loop

import duckdb
import fabduckdb

createstatement = "CREATE or REPLACE TABLE abc as SELECT * FROM range(1,10,1) t(x);CREATE or REPLACE TABLE def as SELECT * FROM range(20,30,2) t(x)"
loopstatement = "create table xyz as (loop (select * from {{x}}) over (select x from (values('abc'), ('def')) t(x)))"
duckdb.default_connection.execute(createstatement)
duckdb.default_connection.execute(loopstatement)
df = duckdb.default_connection.execute("select * from xyz").df()
df

In [ ]:
# Demonstrate a more interesting Jinja2 Template

import duckdb
import fabduckdb

createstatement = "CREATE or REPLACE TABLE abc as SELECT * FROM range(1,10,1) t(y);CREATE or REPLACE TABLE def as SELECT * FROM range(20,30,2) t(y)"

template_statement = """
    {% set val = "'Abc is as easy as 123'" if x == 'abc' else "'I am he who is x, y and zee'"%}
    select 
    '{{x}}', y, {{val}} from {{x}}
"""
loopstatement = f"create or REPLACE table xyz as (loop ({template_statement}) over (select x from (values('abc'), ('def')) t(x)))"

duckdb.default_connection.execute(createstatement)
duckdb.default_connection.execute(loopstatement)
df = duckdb.default_connection.execute("select * from xyz").df()

display(df)

In [ ]:
# Run different SQL depending on value
import duckdb
import fabduckdb

createstatement = "CREATE or REPLACE TABLE abc as SELECT * FROM range(1,10,1) t(y);CREATE or REPLACE TABLE def as SELECT * FROM range(20,30,2) t(y)"

template_statement = """
    {% set val = "select y * 10 as abc123 from " + x if x == 'abc' else "select y * 100 as thisisnotabc from " + x%}
    {{val}}
"""
loopstatement = f"create or REPLACE table xyz as (loop ({template_statement}) over (select x from (values('abc'), ('def')) t(x)) using 'union all by name')"

duckdb.default_connection.execute(createstatement)
duckdb.default_connection.execute(loopstatement)
df = duckdb.default_connection.execute("select * from xyz").df()

display(df)